<h3> Data Analysis with SQL </h3>

<p> Using SQLite3 integration into Jupyter Notebook, we will perform further data analysis with SQL using the additional_info file that contains more useful information about the players. The main advantage of SQL to do further analysis is its easy syntax compared to Python, while Python is more suited with wrangling.  The following cells installs SQLite3 and connects the dataframes into SQL. </p>

In [ ]:
import sqlite3

In [ ]:
!pip install ipython-sql

In [ ]:
cnn = sqlite3.connect('nba_player_analysis.db')

In [ ]:
merged=pd.read_csv("./player_shooting.csv")

In [ ]:
additional_info=pd.read_csv("./additional_info.csv")

In [ ]:
merged.to_sql('player_shooting', cnn)
additional_info.to_sql('additional_info', cnn)

In [ ]:
%load_ext sql 
%sql sqlite:///nba_player_analysis.db

In [ ]:
%%sql 

CREATE temporary TABLE player_data (
name varchar(30),
num_shots int,
real_height float,
release_time float,
release_vel float,
max_vel float, 
left_to_right float,
right_to_left float,
arc float,
raw_arc float,
`release` varchar(100),
`jump_dist` varchar(100),
height int, 
career_fg float,
career_fg_3 float,
college varchar(100),
draft_team varchar(100),
position varchar(100),
hand varchar(100),
weight varchar(50),
`status` int
);

#### Quick Data Cleaning:
- Renaming and selecting necessary columns for analysis
- Joining the shooting data with additional_info table, which contains further details about players

In [ ]:
%%sql
INSERT INTO player_data 
WITH p_stats AS (
	SELECT name, n AS num_shots, ROUND(hght, 1) as real_height, rt AS release_time, rv AS release_vel, mxv AS max_vel, lr1t AS left_to_right, rl1t AS right_to_left, plr AS arc, arc_angle AS raw_arc, `Release` AS `release` , jump_dist, approx_h AS height, `status`
    FROM player_shooting
)
SELECT p.name, num_shots, real_height, release_time, release_vel, max_vel, left_to_right, right_to_left, arc, raw_arc, `release`, jump_dist, height, career_fg, career_fg_3, college, draft_team, 
position, hand, weight, `status` FROM p_stats p
INNER JOIN (SELECT name, `career_FG%` AS `career_fg`, `career_FG3%` AS `career_fg_3`, college, draft_team, `position`, shoots AS hand, weight FROM additional_info) ai
ON p.name = ai.name;

UPDATE player_data 
SET 
    weight = REPLACE(weight, 'lb', '');

ALTER TABLE player_data
ALTER COLUMN weight int;

### Analysis

#### 1. Ranking of field goals per each team and their arcs (and selecting top scorers)

In [ ]:
%%sql 

WITH ranks AS (
SELECT name, career_fg_3, raw_arc, draft_team,
RANK() OVER (PARTITION BY draft_team ORDER BY career_fg_3 DESC) AS fg_rank,
RANK() OVER (PARTITION BY draft_team ORDER BY raw_arc DESC) AS arc_rank
FROM (SELECT * FROM player_data WHERE `status` = 1) p )
SELECT name, fg_rank, arc_rank FROM ranks
WHERE fg_rank BETWEEN 1 AND 2;

#### 2. Comparing the arc, release time and release velocity of different handed players

In [ ]:
%%sql

SELECT 
    AVG(arc) AS avg_arc,
    AVG(release_time) AS avg_rt,
    AVG(release_vel) AS avg_rv,
    hand
FROM
    player_data
GROUP BY hand;

#### 3. Average field goal for each handedness and shoot pocket & selecting top 3 averages

In [ ]:
%%sql

SELECT 
    hand,
    CASE
        WHEN left_to_right > right_to_left THEN 'R'
        WHEN left_to_right < right_to_left THEN 'L'
        WHEN left_to_right = 0 AND right_to_left = 0 THEN 'S'
    END AS shoot_pocket,
    ROUND(AVG(career_fg), 2) AS avg_fg
FROM
    player_data
GROUP BY hand , shoot_pocket
ORDER BY avg_fg DESC
LIMIT 3;

#### 4. Avg metrics for each weightclasses (> 200 lb: heavy, < 170: light, in-between: Normal)

In [ ]:
%%sql

SELECT 
    AVG(arc) AS avg_arc,
    AVG(career_fg) AS avg_fg,
    AVG(release_time) AS avg_rt,
    CASE
        WHEN 170 < weight AND weight < 200 THEN 'Normal'
        WHEN weight < 170 THEN 'Light'
        WHEN weight > 200 THEN 'Very Heavy'
        ELSE 'Light'
    END AS weight_class
FROM
    player_data
GROUP BY weight_class;

#### 5. Average metrics for height classes 

In [ ]:
%%sql

SELECT 
    AVG(arc), AVG(career_fg_3), AVG(release_time), real_height
FROM
    player_data
WHERE
    real_height != 0
GROUP BY real_height;

#### 6. Does jump distance lead to higher FG percentage?

In [ ]:
%%sql

WITH  t_ AS (SELECT *,
DENSE_RANK() OVER(PARTITION BY position ORDER BY jump_dist) AS j_rank FROM player_data),
t__ AS (SELECT career_fg, j_rank, position,
LAG(career_fg, 1) OVER (PARTITION BY position ORDER BY j_rank ASC) AS `prev`
FROM t_)
SELECT DISTINCT position, j_rank, IIF(career_fg - `prev` > 0, "Better", "Not Better") AS better_than_prev
FROM t__;

#### 7. Does release time affect three-point performance?

In [ ]:
%%sql

SELECT 
    AVG(career_fg_3) AS three_pt_fg, `release`
FROM
    player_data
GROUP BY `release`;

#### 8. Which college produces the best scorers?  

In [ ]:
%%sql

SELECT 
    college, AVG(career_fg) AS fg
FROM
    player_data
GROUP BY college
ORDER BY fg DESC;

#### 9. Comparing all metrics between players that made and missed the shot.

REMARK: Every even-th column are made shot metrics and every odd-th was missed.

In [ ]:
%%sql

WITH tt as (
SELECT *,
LEAD(release_time,1) OVER lead_window AS rt_win,
LEAD(release_vel,1) OVER lead_window AS rv_win,
LEAD(max_vel,1) OVER lead_window AS mv_win,
LEAD(left_to_right,1) OVER lead_window AS lr_win,
LEAD(right_to_left,1) OVER lead_window AS rl_win,
LEAD(arc,1) OVER lead_window AS arc_win
FROM player_data 
WINDOW lead_window AS (partition by name))
SELECT * FROM (select name, ROUND(release_time - ifnull(rt_win, null), 1) as rt_diff,
ROUND(release_vel - ifnull(rv_win, null), 1) as rv_diff,
ROUND(max_vel - ifnull(mv_win, null), 1) as mv_diff,
ROUND(left_to_right - ifnull(lr_win, null), 1) as lr_diff,
ROUND(right_to_left - ifnull(rl_win, null), 1) as rl_diff,
ROUND(arc - ifnull(arc_win, null), 1) as arc_diff
FROM tt) tt_
WHERE rt_diff IS NOT NULL
LIMIT 20;

##### 10. Between made vs missed shots, what are the differences in how much you "pull" back when you release?

In [ ]:
%%sql

SELECT status, AVG(ry) pull_back FROM player_shooting
GROUP BY status
ORDER BY AVG(ry);

## Conclusion

#TODO